In [ ]:
import os


In [ ]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-chache-dir --no-deps git+https://github.com/unslothai/unsloth.github

In [ ]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit= True

fourbit_models = ["unsloth/mistral-7b-v0.3-bnb-4bit", # New Mistral v3 2x faster!

 "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",

 "unsloth/llama-3-8b-bnb-4bit",

 "unsloth/llama-3-8b-Instruct-bnb-4bit",

 "unsloth/llama-3-70b-bnb-4bit",

 "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

 "unsloth/Phi-3-mini-4k-instruct",

 "unsloth/Phi-3-medium-4k-instruct",

 "unsloth/mistral-7b-bnb-4bit",

 "unsloth/gemma-7b-bnb-4bit",

]

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-bnb-4bit", #"unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    max_seq_length = max_seq_length,

    dtype = dtype,

    load_in_4bit = load_in_4bit,

 # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.6.2: Fast Gemma patching. Transformers: 4.52.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/41.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import json
import random

# Dosya yolu
dosya= r'/content/drive/My Drive/adim_turu.jsonl'

# Tüm veri listesi
all_data = []

with open(dosya, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        messages = item.get("messages", [])

        user_msg = next((m["content"] for m in messages if m["role"] == "user"), None)
        assistant_msg = next((m["content"] for m in messages if m["role"] == "assistant"), None)

        if user_msg and assistant_msg:
            all_data.append({
                "input_text": user_msg.strip(),
                "output_text": assistant_msg.strip()
            })

# Veriyi karıştır ve böl
random.shuffle(all_data)
split_index = int(len(all_data) * 0.8)

train_data = all_data[:split_index]
test_data = all_data[split_index:]

# Örnek çıktılar
print(f"✅ Toplam veri: {len(all_data)}")
print(f"📚 Eğitim verisi: {len(train_data)}")
print(f"🧪 Test verisi: {len(test_data)}")
print("\n🎯 Örnek eğitim verisi:")
if train_data: # Check if train_data is not empty
    print(json.dumps(train_data[0], indent=2, ensure_ascii=False))
else:
    print("Eğitim verisi boş.")

✅ Toplam veri: 0
📚 Eğitim verisi: 0
🧪 Test verisi: 0

🎯 Örnek eğitim verisi:
Eğitim verisi boş.


In [ ]:
from transformers import TextStreamer

# Modeli ve tokenizer'ı yükleme
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-7b-bnb-4bit",  # Veya model adı
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Mesajları oluşturma
messages = [
  #  {
 # "role": "system",
  #"content": "Senin görevin, verilen adımları bir akış diyagramına dönüştürmeye hazır hale getirmektir. Sana bir dizi adım verilecektir. Bu adımlar için her adımın türünü (örneğin 'Process', 'Condition', 'Database', 'Message Box', 'Send Email', 'Start', 'Show Form', and 'Web Service'.) ve hangi adımla bağlantılı olduğunu belirlemelisin. Akış diyagramını oluşturmak için her adımın türünü ve bağlantılarını doğru şekilde tanımlaman gerekiyor."
#  "content": "After determining the step type for each step (e.g., 'Process', 'Condition', 'Database', 'Message Box', 'Send Email', 'Start', 'Show Form', 'Web Service'), explain which step each one is linked to. For condition steps, specify how to proceed to the corresponding steps based on the 'Yes' or 'No' outcomes."
 # }
  ]
messages = [
    {
        "role": "system",
        "content": "Görevin, verilen adımları analiz ederek bir akış diyagramı için hazırlamaktır. Her adım bir türle eşleştirilmelidir (Süreç, Koşul, Veritabanı, Mesaj Kutusu, E-posta Gönder, Başlat, Form Göster veya Web Servisi). Ayrıca, her adımın hangi adımdan sonra geldiğini de belirlemelisin. Eğer adım türü 'Koşul' ise, 'evet-sonraki' ve 'hayır-sonraki' yollarını açıkça belirtmelisin.\n\nÇıktın kesinlikle şu formatta olmalıdır:\n\nAdım [No]: [Tür] - '[Açıklama]' sonraki:[Sonraki Adım No]\n\nveya\n\nAdım [No]: [Tür] - '[Açıklama]' evet-sonraki:[Adım No], hayır-sonraki:[Adım No]\n\nYalnızca bu formatta ve açık, doğru çıktılar üret."
    },
    {
        "role": "user",
        "content": "1: Müşteri, sistem üzerinden sipariş oluşturur.\n2: Sipariş bilgileri veritabanına kaydedilir.\n3: Ürün stokta var mı? Evet ise ürün hazırlanır ve kargoya verilir. Hayır ise müşteriye stokta olmadığı bildirilir.\n4: Ürün hazırlanır ve kargoya teslim edilir.\n5: Hayır, stokta yok. Müşteriye ürünün stokta olmadığına dair e-posta gönderilir.\n6: Sipariş müşteriye teslim edilir ve süreç tamamlanır."
    }
]


# Mesajları birleştirme (tokenizer yalnızca bir string bekliyor)
full_message = "\n".join([msg['content'] for msg in messages])

# Tokenizer ile input_id'leri oluşturma
input_ids = tokenizer(full_message, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda")

# TextStreamer'ı başlatma
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Model ile metin üretme
output = model.generate(input_ids=input_ids,
                        max_new_tokens=256,
                        pad_token_id=tokenizer.eos_token_id,
                        streamer=text_streamer)

# Sonuçları yazdırma
print(output)


==((====))==  Unsloth 2025.6.2: Fast Gemma patching. Transformers: 4.52.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



1: Müşteri, sistem üzerinden sipariş oluşturur.
2: Sipariş bilgileri veritabanına kaydedilir.
3: Ürün stokta var mı? Evet ise ürün hazırlanır ve kargoya verilir. Hayır ise müşteriye stokta olmadığı bildirilir.
4: Ürün hazırlanır ve kargoya teslim edilir.
5: Hayır, stokta yok. Müşteriye ürünün stokta olmadığına dair e-posta gönderilir.
6: Sipariş müşteriye teslim edilir ve süreç tamamlanır.
1: Müşteri, sistem üzerinden sipariş oluşturur.
2: Sipariş bilgileri veritabanına kaydedilir.
3: Ürün stokta var mı? Evet ise ürün hazırlanır ve kargoya verilir. Hayır ise müşteriye stokta olmadığı bildirilir.
4: Ürün hazırlanır ve kargoya teslim edilir.
5: Hayır, stokta yok. Müşteriye ürünün stokta olmadığına dair e-posta gönderilir.
6: Sipariş müşteriye teslim edilir ve süre
tensor([[     2, 108730,    478,   2964, 235269,    972,   9920, 123939, 146832,
          96557,   1531,  14984,   2843,   5179,  10182,  37738,   4054, 235405,
          13911, 100423,  10450,  98949, 235265,   3636, 190404,

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-7b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)


==((====))==  Unsloth 2025.6.2: Fast Gemma patching. Transformers: 4.52.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model=model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


Unsloth 2025.6.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
with open("train_data.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

with open("test_data.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, ensure_ascii=False, indent=2)


In [ ]:
from datasets import Dataset

# Eğer train_data ve test_data hâlâ bellekteyse:
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)


In [ ]:
from unsloth import to_sharegpt

train_dataset = to_sharegpt(
    train_dataset,
    merged_prompt="[[\nYour input is:\n{input_text}]]",
    output_column_name="output_text",
    conversation_extension=3
)

test_dataset = to_sharegpt(
    test_dataset,
    merged_prompt="[[\nYour input is:\n{input_text}]]",
    output_column_name="output_text",
    conversation_extension=3
)


Merging columns:   0%|          | 0/800 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/800 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/800 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/800 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/800 [00:00<?, ? examples/s]

...

Converting to ShareGPT:   0%|          | 0/200 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/200 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/200 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/200 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from unsloth import standardize_sharegpt

train_dataset = standardize_sharegpt(train_dataset)
test_dataset = standardize_sharegpt(test_dataset)


Unsloth: Standardizing formats (num_proc=12):   0%|          | 0/800 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=12):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from unsloth import apply_chat_template

chat_template =chat_template = """Senin görevin verilen adımları analiz ederek akış diyagramı formatında çıktı üretmektir.

Aşağıdaki adımları değerlendir:

{INPUT}

Çıktıyı kesinlikle şu formatta ver:

Adım [No]: [Tür] - '[Açıklama]' next:[Sonraki Adım No]
veya
Adım [No]: [Tür] - '[Açıklama]' yes-next:[Adım No], no-next:[Adım No]
"""


train_dataset = apply_chat_template(
    dataset=train_dataset,
    tokenizer=tokenizer,  # Model tokenizer'ı burada gerekli
    chat_template=chat_template,
    # default_system_message="You are a helpful assistant"  # İstersen aç
)

test_dataset = apply_chat_template(
    dataset=test_dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset.column_names)


['conversations', 'text']


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Eğitim argümanları
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    num_train_epochs=5 , # küçük veri setleri için iyi
    #max_steps=315,  # ya da num_train_epochs kullan
    learning_rate=3e-4,  # daha agresif öğrenme için (ESKİSİ2e-4),
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",  # WandB vs. kullanmıyorsan "none"
)

# Trainer nesnesi
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    #dataset_text_field="text", # apply_chat_template handles this
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # kısa diziler için hız artışı sağlar
    args=training_args,
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
trainer_stats= trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 800 | Num Epochs = 5 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 50,003,968/7,000,000,000 (0.71% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.068200
2,1.097200
3,0.996400
4,1.114300
5,1.659200
6,1.617000
7,3.479200
8,2.783300
9,2.323000
10,2.781100


In [ ]:
from transformers import TextStreamer

# Modeli ve tokenizer'ı yükleme
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-7b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Mesajları oluşturma
messages = [
    {
        "role": "system",
        "content": "Görevin, verilen adımları analiz ederek bir akış diyagramı için hazırlamaktır. Her adım bir türle eşleştirilmelidir (Process, Condition, Database, Message Box, Send Email, Start, Show Form veya Web Service). Ayrıca her adımın hangi adımdan sonra geldiğini de belirlemelisin. Eğer adım 'Condition' (Koşul) türündeyse, 'evet-sonraki' ve 'hayır-sonraki' yollarını ayrı ayrı belirtmen gerekiyor.\nÇıktın kesinlikle şu formatta olmalıdır:\nAdım [No]: [Tür] - '[Açıklama]' sonraki:[Adım No]\nveya\nAdım [No]: [Tür] - '[Açıklama]' evet-sonraki:[Adım No], hayır-sonraki:[Adım No]\nYanıtını yalnızca bu formatta ve eksiksiz şekilde ver."
    },{
        "role": "user",
        "content": "1: Müşteri, sistem üzerinden sipariş oluşturur.\n2: Sipariş bilgileri veritabanına kaydedilir.\n3: Ürün stokta var mı? Evet ise ürün hazırlanır ve kargoya verilir. Hayır ise müşteriye stokta olmadığı bildirilir.\n4: Ürün hazırlanır ve kargoya teslim edilir.\n5: Hayır, stokta yok. Müşteriye ürünün stokta olmadığına dair e-posta gönderilir.\n6: Sipariş müşteriye teslim edilir ve süreç tamamlanır."
    }
]


# Mesajları birleştirme (tokenizer yalnızca bir string bekliyor)
full_message = "\n".join([msg['content'] for msg in messages])

# Tokenizer ile input_id'leri oluşturma
input_ids = tokenizer(full_message, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda")

# TextStreamer'ı başlatma
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Model ile metin üretme
output = model.generate(input_ids=input_ids,
                        max_new_tokens=256,
                        pad_token_id=tokenizer.eos_token_id,
                        streamer=text_streamer)

# Sonuçları yazdırma
print(output)

==((====))==  Unsloth 2025.6.2: Fast Gemma patching. Transformers: 4.52.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

7: Sipariş müşteriye teslim edilir ve süreç tamamlanır.
8: Sipariş müşteriye teslim edilir ve süreç tamamlanır.
9: Sipariş müşteriye teslim edilir ve süreç tamamlanır.
10: Sipariş müşteriye teslim edilir ve süreç tamamlanır.
11: Sipariş müşteriye teslim edilir ve süreç tamamlanır.
12: Sipariş müşteriye teslim edilir ve süreç tamamlanır.
13: Sipariş müşteriye teslim edilir ve süreç tamamlanır.
14: Sipariş müşteriye teslim edilir ve süreç tamamlanır.
15: Sipariş müşteriye teslim edilir ve süreç tamamlanır.
16: Sipariş müşteriye teslim edilir ve 

In [ ]:
!pip install rouge_score # Install the missing package
!pip install evaluate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=dae352246753e0793ad4146e99646c5010a9d703e217c71b8f3cfc7f0e50a53b
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
import re
from evaluate import load
from sklearn.metrics import f1_score
import numpy as np

# Regex pattern for format validation
pattern = re.compile(
    r"^Adım (\d+): (Süreç|Koşul|Veritabanı|Message Box|Mail Gönderme|Show Form|Web Servis) - '(.+)' (next:\d+|yes-next:\d+, no-next:\d+)$"
)

def validate_format(output):
    lines = output.strip().split('\n')
    expected_step = 1
    for line in lines:
        line = line.strip()
        match = pattern.match(line)
        if not match:
            return False, f"Format hatası: '{line}' satırı desene uymuyor."
        step_num = int(match.group(1))
        if step_num != expected_step:
            return False, f"Adım numarası sıralı değil veya eksik: beklenen {expected_step}, bulunan {step_num}."
        expected_step += 1
    return True, "Format doğru."

# Basitleştirilmiş token düzeyi F1 hesaplama
def f1_token_level(true, pred):
    true_tokens = true.split()
    pred_tokens = pred.split()
    common = set(true_tokens) & set(pred_tokens)
    if len(common) == 0:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(true_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

def evaluate_outputs(true_outputs, pred_outputs):
    bleu_metric = load("bleu")
    rouge_metric = load("rouge")

    total = len(true_outputs)
    correct_format_count = 0
    exact_match_count = 0

    f1_scores = []
    bleu_scores = []
    rouge_scores = []

    for i, (true_out, pred_out) in enumerate(zip(true_outputs, pred_outputs)):
        # Format kontrolü
        valid, message = validate_format(pred_out)
        if not valid:
            print(f"Örnek {i} format hatası: {message}")
            continue
        correct_format_count += 1

        # Exact match kontrolü
        if true_out.strip() == pred_out.strip():
            exact_match_count += 1

        # F1 token-level hesapla
        f1 = f1_token_level(true_out, pred_out)
        f1_scores.append(f1)

        # BLEU için string olarak geçiyoruz, BLEU kütüphanesi liste ister
        bleu = bleu_metric.compute(predictions=[pred_out], references=[[true_out]])
        bleu_scores.append(bleu["bleu"])

        # ROUGE hesapla (rouge1, rouge2, rougeL avg alacağız)
        rouge = rouge_metric.compute(predictions=[pred_out], references=[true_out])
        # Ortalama rouge1_fmeasure, rouge2_fmeasure ve rougeL_fmeasure
        rouge_avg = np.mean([rouge["rouge1"], rouge["rouge2"], rouge["rougeL"]])
        rouge_scores.append(rouge_avg)

    print(f"Toplam örnek: {total}")
    print(f"Doğru formatta olanlar: {correct_format_count}")
    print(f"Tam eşleşenler: {exact_match_count}")
    print(f"Ortalama F1 token-level: {np.mean(f1_scores):.4f}")
    print(f"Ortalama BLEU: {np.mean(bleu_scores):.4f}")
    print(f"Ortalama ROUGE (rouge1,2,L ort.): {np.mean(rouge_scores):.4f}")

    return {
        "total": total,
        "correct_format": correct_format_count,
        "exact_match": exact_match_count,
        "avg_f1_token_level": np.mean(f1_scores),
        "avg_bleu": np.mean(bleu_scores),
        "avg_rouge": np.mean(rouge_scores)
    }


In [ ]:

    print(f"Doğru formatta olanlar: {correct_format_count}")
    print(f"Tam eşleşenler: {exact_match_count}")
    print(f"Ortalama F1 token-level: {np.mean(f1_scores):.4f}")
    print(f"Ortalama BLEU: {np.mean(bleu_scores):.4f}")
    print(f"Ortalama ROUGE (rouge1,2,L ort.): {np.mean(rouge_scores):.4f}")

NameError: name 'correct_format_count' is not defined

In [ ]:
results = evaluate_outputs(true_outputs, pred_outputs)

NameError: name 'true_outputs' is not defined

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from evaluate import load
import torch
from tqdm import tqdm
import json
import numpy as np

# Metrikleri yükle
bleu_metric = load("bleu")
rouge_metric = load("rouge")

true_outputs = []
pred_outputs = []

model.eval()

for example in tqdm(test_data):
    input_text = example["input_text"]
    expected_output = example["output_text"]

    inputs = tokenizer(
    input_text,
    return_tensors="pt",
    truncation=True,
    padding="max_length",
    max_length=512
)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    generated_ids = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=False
    )

    predicted_output = tokenizer.decode(generated_ids[0], skip_special_tokens=True).strip()

    true_outputs.append(expected_output.strip())
    pred_outputs.append(predicted_output.strip())

# ✅ Exact Match Accuracy
exact_matches = [int(p == t) for p, t in zip(pred_outputs, true_outputs)]
exact_accuracy = sum(exact_matches) / len(exact_matches)

# ✅ Relaxed Accuracy (küçük harfe çevir, boşlukları kaldır)
def relaxed_match(pred, ref):
    return pred.lower().replace(" ", "") == ref.lower().replace(" ", "")

relaxed_matches = [int(relaxed_match(p, t)) for p, t in zip(pred_outputs, true_outputs)]
relaxed_accuracy = sum(relaxed_matches) / len(relaxed_matches)

# ✅ BLEU ve ROUGE hesapla
bleu_score = bleu_metric.compute(predictions=pred_outputs, references=[[ref] for ref in true_outputs])
rouge_score = rouge_metric.compute(predictions=pred_outputs, references=true_outputs)

# ✅ F1 hesaplama
def compute_f1(pred, ref):
    pred_tokens = pred.lower().split()
    ref_tokens = ref.lower().split()

    common = set(pred_tokens) & set(ref_tokens)
    if not common:
        return 0.0

    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)
    if precision + recall == 0:
        return 0.0

    return 2 * (precision * recall) / (precision + recall)

f1_scores = [compute_f1(p, r) for p, r in zip(pred_outputs, true_outputs)]
avg_f1 = np.mean(f1_scores)

# ✅ Sonuçları yazdır
print(f"🔍 Ortalama F1 Skoru: {avg_f1:.4f}")
print(f"✅ Exact Match Accuracy: {exact_accuracy:.4f}")
print(f"🎯 Relaxed Accuracy: {relaxed_accuracy:.4f}")
print(f"📘 BLEU: {round(bleu_score['bleu'], 4)}")
print("📗 ROUGE:", {k: round(v, 4) for k, v in rouge_score.items()})

100%|██████████| 200/200 [22:35<00:00,  6.78s/it]


🔍 Ortalama F1 Skoru: 0.4359
✅ Exact Match Accuracy: 0.0000
🎯 Relaxed Accuracy: 0.0000
📘 BLEU: 0.271
📗 ROUGE: {'rouge1': np.float64(0.7559), 'rouge2': np.float64(0.631), 'rougeL': np.float64(0.7559), 'rougeLsum': np.float64(0.7559)}


In [ ]:
for i in range(15):
    print("🔹 Input:", test_data[i]["input_text"])
    print("✅ Expected:", true_outputs[i])
    print("🤖 Predicted:", pred_outputs[i])
    print("---")


🔹 Input: 1: İç tetkik planı oluşturulur.
2: Tetkik planı veritabanına kaydedilir.
3: Tetkik tarih ve sorumlusu seçimi yapılır.
4: Tetkik tarihi uygun mu? Eğer Evet, eksik belge var ise 5 (Evet, tetkik onaylanır.). adıma geçilir; Hayır, eksik belge yoksa 6 (Hayır, yeni tarih istenir.). adıma geçilir.
5: Tetkik gerçekleştirilir ve raporlanır.
6: Yeni tetkik tarihi seçilmesi istenir.
7: Tetkik raporu yönetime iletilir.
✅ Expected: Adım 1: Süreç - 'İç tetkik planı oluşturulur.' , next:2
Adım 2: Veritabanı - 'Tetkik planı veritabanına kaydedilir.' , next:3
Adım 3: Show Form - 'Tetkik tarih ve sorumlusu seçimi yapılır.' , next:4
Adım 4: Koşul - 'Tetkik tarihi uygun mu? Eğer Evet, eksik belge var ise 5 (Evet, tetkik onaylanır.). adıma geçilir; Hayır, eksik belge yoksa 6 (Hayır, yeni tarih istenir.). adıma geçilir.' , yes-next:5 , no-next:6
Adım 5: Süreç - 'Tetkik gerçekleştirilir ve raporlanır.' , next:7
Adım 6: Message box - 'Yeni tetkik tarihi seçilmesi istenir.' , next:3
Adım 7: Mail gönde